Important Libraries

In [1]:
#read docx file
!pip install python-docx
import docx
from docx import Document
from nltk.stem.isri import ISRIStemmer

import pandas as pd
import numpy as np

import re
import math


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=530dcfbc44faf8b93de992f9b955451e74b22e0d833bbf806043a06f69763bfe
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


Functions for Getting Information

In [2]:
from collections import defaultdict

inverted_index = {}

with open('term_index.txt', 'r') as lines:
    
    #iterate in every line
    for line in lines:

      #store the previous doc ids
      temp=-1
      temp1=-1

      #split the line on the basis of space
      for index, data in enumerate(line.strip().split(' ')):

        if index == 0:

          #get the document id
          id=int(data)

          #for every document id we have to crate the list
          inverted_index[id] = defaultdict(list)
        else:


          flag=True

          #split on the basis of : and then check the length
          if len(data.split(":")) == 1:
            flag=False
            position = int(data.split(":")[0])
          else:

            #get the document id
            doument_id = int(data.split(":")[0])      

            #get the psotions         
            position = int(data.split(":")[1])

          #if temp = -1 means we didnot have stored any thing yet
          if temp == -1:

            #append the positios in the dixtionary
            inverted_index[id][doument_id].append(position)
            temp = doument_id
            temp1 = position
          else:

            #if the flag is false 
            if flag == False:
              temp1 += position
              inverted_index[id][temp].append(temp1)
            else:

              #store the document id in the temp variable for the future use
              temp += doument_id

              #store the positions in the dictionary
              inverted_index[id][temp].append(position)      

In [4]:
# Initialize an empty dictionary
term_info_dict = {}

# Open the file for reading
with open("/content/term_info (2).txt", "r") as file:
    # Iterate over each line in the file
    for line in file:
        # Split the line using the "/" separator
        line_items = line.strip().split("/")
        # Extract the key and values
        key = int(line_items[0])
        values = [int(value) for value in line_items[1:]]
        # print('key', key, 'value:', values)
        # Add the key and values to the dictionary
        term_info_dict[key] = values
        
doc_index = {}
with open('doc_index.txt', 'r',encoding='utf-8') as file:

    for line in file:
        line=line.strip().split()

        #line[0] is the document id
        #if the document id isnot present in the dictionary then add it
        if line[0] not in doc_index:
            doc_index[line[0]] = len(line)-2
        else:
            doc_index[line[0]]+= len(line)-2


In [11]:

docids = {}

#open the docids.txt file
with open("docids.txt", 'r') as f:
    for line in f:
        docid, path = line.strip().split('/')
        docids[int(docid)] = path

# Load the Word document
document = Document('queries.docx')
stop_words_set = set(pd.read_csv('/content/Urdu stopwords.txt', squeeze=True, header=None))


# Get the first table in the document
table = document.tables[0]


def remove_non_urdu(text):
    urdu_alphabets = u'\u0600-\u06FF'

    # Remove all the non urdu alphabets and maintain the spaces
    text = re.sub('[^%s]' % urdu_alphabets, ' ', text)

    # Remove , : ; . from the text
    text = re.sub('[,.;:]', '', text)

    return text


# Create an empty dictionary to store the data
data_dict = {}

# Initialize the keys variable to None
keys = None

# Iterate through each row in the table
for i, row in enumerate(table.rows):
    
    # Get the text content of each cell in the row
    text = [cell.text.strip() for cell in row.cells]
    
    # If this is the first row, save the values as the keys for the dictionary
    if i == 0:
        keys = tuple(text)
        continue
    
    # If this is not the first row, create a new dictionary with the values in the row
    key = text[0]
    #print(key, 'j')

    values = list(zip(keys[1:], text[1:]))

    topics=text[2]
    unwanat=text[3]

    unwanat=remove_non_urdu(unwanat)
    u=unwanat.split()

    stemmer = ISRIStemmer()
    term_id_dict = {}
    
    #remove stop words
    for i in u:

        if i in stop_words_set:
            u.remove(i)
    # print(u)
    #after removing stop words
    values.append(u)

    # Add the new dictionary to the data dictionary using the key from the first cell as the key
    data_dict[key] = values

term_id_dict={}

# Print the resulting dictionary
# print(data_dict)

for i in data_dict:

    # query
    query=data_dict[i][3] 

    with open('termids.txt', 'r') as f:
        for j in f:
            term, term_id= j.strip().split('/')
            
            for q in query:
              
                if q == term:
                    term_id_dict[term]=term_id

<ipython-input-11-6a04628aa24d>:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  stop_words_set = set(pd.read_csv('/content/Urdu stopwords.txt', squeeze=True, header=None))


Function no 1: TF-IDF

In [12]:

tf_idf_for_query={}
term_frequency_for_query=0
query_id=1

for i in data_dict:

    # query
    query=data_dict[i][3] 

    for j in query:

        if len(j)==0:
            continue

        if j== 'دی':
            continue

        #get the term id from the dictionary we have created
        term_id=int(term_id_dict[j])

        term_frequency_for_query+=query.count(j)
        
        df=term_info_dict[term_id][2]

        #if the freq of the gievn word is zero
        if term_frequency_for_query==0:
            term_frequency_for_query=1
        else:
            term_frequency_for_query=1+np.log10(term_frequency_for_query)

        #no of documnets
        n=5771

        #idf for the given term
        idf=1+np.log10(n/df)

        #get the tf idf for the query
        tf_idf_for_query[query_id, term_id]=term_frequency_for_query*idf

    query_id+=1

In [13]:
#dictionary and variable for the tf
document_tf={}
term_frequency = 0

#in the data dictionary we have the data of the queries
for key in data_dict:

    term_frequency=0

    #extract the query from the data dictioanry which has splitted query
    query = data_dict[key][3]

    for term in query:

        term_frequency=0


        if term == 'دی':
            continue

        #if the term length is zero then we have to avoid
        if len(term) == 0:
            continue

        #find term id of the term
        termid = term_id_dict[term]

        #iterate through the list of document ids
        for docid in docids:
            #for every documnent count the number of times the term appears
            term_frequency = len(inverted_index[int(termid)][docid])
            
            #as trhe term frequency of zero should be one after normalization
            if term_frequency == 0:

                #term frequncy = 1
                term_frequency = 1
            else:

                #normalize the term frequency 
                term_frequency = 1 + np.log10(term_frequency)

            #store the query # term # and tf-idf score in the dictionary
            document_tf[docid,termid]=term_frequency


In [14]:

# create a dictionary to store the cosine similarity between each query and document
cosine_similarity_dictionary = {}

# calculate cosine similarity between each query and document
for query_key, query_tf_idf in tf_idf_for_query.items():

    #we have the query_id and term_id for the elements of the query
    query_id, _ = query_key

    #create a list at every query_id
    cosine_similarity_dictionary[query_id] = defaultdict(int)
  
    for doc_key, doc_tf in document_tf.items():
    
        #document_id and term id for the doumnent 
        doc_id, _ = doc_key

        #dot product
        dot_product = 0.0
      
        # calculate dot product and magnitude of query vector
        dot_product += query_tf_idf * doc_tf
    
        #if query_id and doc_id already exists
        if query_id in cosine_similarity_dictionary and doc_id in cosine_similarity_dictionary[query_id]:
            cosine_similarity_dictionary[query_id][doc_id] = dot_product 
        else:

            #add the dot product to query
            cosine_similarity_dictionary[query_id][doc_id] += dot_product 


Function no 2 : OKAPA BM25

In [15]:
#find the average document length
average_length = 0
for key in doc_index:
    average_length += doc_index[key]


no_of_docs=5771
avg_doc_len=average_length/no_of_docs

#calculate the okapi bm25 score
k=1.2
b=0.75
okapi_bm25_dic = {}

query_id=1
for key in data_dict:
        
        #create the okapi dictionary for every query
        okapi_bm25_dic[query_id] = defaultdict(int)

        term_frequency=0

        #get the query
        query = data_dict[key][3]
    
        for term in query:

            term_frequency=0

            if term == 'دی':
                continue
            if len(term) == 0:
                continue


            #find term id of the term
            term_id = term_id_dict[term]

            #get the document frequency of the term
            document_freq = term_info_dict[int(term_id)][2]

            #iterate through the list of document ids
            for docid in docids:
                
                #for every document get the tf of the term with the document
                term_frequency=document_tf[docid,term_id]

                #get the term frequency for the query
                term_freq_for_query = query.count(term)

                if term_freq_for_query==0:
                    term_freq_for_query=1
                else:

                    #term frequncy for the query
                    term_freq_for_query = 1 + np.log10(term_freq_for_query)

                a=(k*(1-b+b*doc_index[str(docid)]/avg_doc_len)+term_frequency)  *  (k+1)*term_freq_for_query / (k+term_freq_for_query)
                
                #calculte the okapi bm 25 using the formula already given 
                okapi_bm25=  np.log10(5771+0.5 /  document_freq+0.5)  *  (k+1)*term_frequency / a

                #if the queryid and the document id are not already avaliable
                if query_id and docid not in okapi_bm25_dic:
                    okapi_bm25_dic[query_id][docid] = okapi_bm25

                else:

                    #if it is already present then add to the previous values
                    okapi_bm25_dic[query_id][docid] += okapi_bm25
            
        query_id+=1


FUCNTION NO 3:Dirichlet Smoothing

In [16]:
#calculate the total number of terms in the corpus
total_terms = 0

#iterate in the doc index and calculate the total terms
for i in doc_index:
    total_terms += doc_index[i]

#calculate the language model with dirichlet smoothing
dirichlet_smoothing_dic = {}
query_id=1

for key in data_dict:
        

        dirichlet_smoothing_dic[query_id] = defaultdict(int)
        query = data_dict[key][3]
        
        #for every term in the query
        for term in query:
            if term == 'دی':
                continue

            if len(term) == 0:
                continue

            #find term id of the term
            term_id = term_id_dict[term]

            #get the document frequency of the term
            df = term_info_dict[int(term_id)][2]

            #A is the average length
            A=average_length
            
            #get the total frequency of the term in the corpus
            total_tf = term_info_dict[int(term_id)][1]

            #iterate through the list of document ids
            for docid in docids:
                #for every document get the tf of the term with the document
                tf=document_tf[docid,term_id]

                #get the length of this document
                N = doc_index[str(docid)]

                #calculate the language model with dirichlet smoothing
                dirichlet_smoothing = np.log10((N / (N + A)) * (tf / N))  + ((A / (N + A)) * (total_tf/(A*5771)))

                if query_id and docid not in dirichlet_smoothing_dic:
                    dirichlet_smoothing_dic[query_id][docid] = dirichlet_smoothing

                else:

                    #if it is already available then add
                    dirichlet_smoothing_dic[query_id][docid] += dirichlet_smoothing
        query_id+=1


Sorting

In [17]:
sorted_cosine = defaultdict(list)

for query_id in data_dict:
  
  #get the query id and convert it into integer
  query_id=int(query_id)

  #for every document present in the document ids
  for docid in docids:

    sorted_cosine[query_id].append(f"{docid}:{cosine_similarity_dictionary[query_id][docid]}")


#iterate
for i in sorted_cosine:

  #sort 
  sorted_cosine[i] = sorted(sorted_cosine[i], key=lambda i: float(i.split(':')[1]), reverse=True)

  

In [18]:
sorted_okapi = defaultdict(list)

for query_id in data_dict:
  
  #get the query id and convert it into integer
  query_id=int(query_id)

  #for every document present in the document ids
  for docid in docids:

    sorted_okapi[query_id].append(f"{docid}:{okapi_bm25_dic[query_id][docid]}")


#iterate
for i in sorted_cosine:

  #sort 
  sorted_okapi[i] = sorted(sorted_cosine[i], key=lambda i: float(i.split(':')[1]), reverse=True)

  

In [19]:

#sorting the dirichlet_smoothing_dic

#first add
sorted_dirch = defaultdict(list)
for query_id in data_dict:
  
  #get the query id and convert it into integer
  query_id=int(query_id)

  #for every document present in the document ids
  for docid in docids:
    sorted_dirch[query_id].append(f"{docid}:{dirichlet_smoothing_dic[query_id][docid]}")

#iterate and sort
for i in sorted_dirch:
  sorted_dirch[i] = sorted(sorted_dirch[i], key=lambda i: float(i.split(':')[1]), reverse=True)

QRELS FILE 

In [20]:
doc_ids = {}
with open("docids.txt", 'r') as file:
  for line in file:
    value, key = line.strip().split('/')
    doc_ids[int(key.split('.')[0])] = int(value)

In [22]:
file = pd.ExcelFile("qrels.xlsx")

#get the names
sheet_names = file.sheet_names

#qrels dictionary
qrels = {}

#iterate in the list of names
for i in sheet_names:
    qrels[i] = file.parse(i)


qrel = {}

for i, query_id in zip(qrels.keys(), data_dict):

  qrel[query_id] = defaultdict(int)

  #iaterate and get the document id
  for docid in [int(x) for x in qrels[i]['Doc Id'] if int(x) in doc_ids.keys()]:

    #add to the qrel dictionayr
    qrel[query_id][doc_ids[docid]] = int(qrels[i].loc[qrels[i]['Topic Id'] == int(query_id)].loc[qrels[i]['Doc Id'] == int(docid) ]['Doc Relevancy'])



In [23]:
#evaluate the model
def evaluate(hello):

  #precision at 5
  prec_at_5 = defaultdict(list)

  #precision at 10
  prec_at_10 = defaultdict(list) 

  #precision at 20
  prec_at_20 = defaultdict(list)

  #precision at 30
  prec_at_30 = defaultdict(list)

  #iterate through the queries
  for q in data_dict:

    #precision at 5
    prec_at_5[int(q)] = [int(hello[int(q)][i].strip().split(':')[0]) for i in range(5)]

    #precision at 10
    prec_at_10[int(q)] = [int(hello[int(q)][i].strip().split(':')[0]) for i in range(10)]

    #precision at 20
    prec_at_20[int(q)] = [int(hello[int(q)][i].strip().split(':')[0]) for i in range(20)]

    #precision at 30
    prec_at_30[int(q)] = [int(hello[int(q)][i].strip().split(':')[0]) for i in range(30)]

  #return the precision at 5, 10, 20, 30
  return [prec_at_5, prec_at_10, prec_at_20, prec_at_30]

#evaluate the cosine similarity model
evaluation_cosine = evaluate(sorted_cosine)

#evaluate the okapi bm25 model
evaluation_okapi = evaluate(sorted_okapi)

#evaluate the language model with dirichlet smoothing
evaltion_dirch = evaluate(sorted_dirch)


final = [evaluation_cosine,evaluation_okapi,evaltion_dirch]

In [24]:
# iterate over the final list and calculate the precision at k
for i, evalation in enumerate(final):

  for j, p in enumerate(evalation):

    # iterate over the 10 queries
    for k, queryid in enumerate(data_dict):

      #relevance value
      relevance = 0
      non_relevance = 0

      for docids in p[int(queryid)]:
        if qrel[queryid][docids] > 2:
          relevance += 1
        else:
          non_relevance += 1

      temp=relevance/(relevance+non_relevance)
      # calculate the precision at 
      final[i][j][k+1].append(temp)

average_precision = {}

#map function
def map(score, id):

  #avd_p is a list
  average_temp = []

  #query number
  for query_id in data_dict:

    #make a dictionary for each query
    average_precision[id] = defaultdict(int)

    #make a dictionary for each query
    average_precision[id][int(query_id)] = defaultdict(int)

    #relevance value
    relevance = 0

    #sum
    sumd = 0

    #for each document in the query
    for ind, vals in enumerate(score[int(query_id)]):

      #get the document id
      docid = int(vals.strip().split(':')[0])

      #if the document is relevant
      if qrel[query_id][docid] > 2:

        #increment the relevance value
        relevance += 1

        #calculate the average precision
        average_precision[id][int(query_id)][ind] = relevance/(ind+1)

        #add the average precision to the sum
        sumd += average_precision[id][int(query_id)][ind]

    #append the average precision to the list
    b=sumd/relevance
    average_temp.append(b)

  a = sum(average_temp)/len(data)

  #return the mean average precision
  return a

#calculate the mean average precision for each model
map1 = map(sorted_cosine, 0)

#calculate the mean average precision for each model
map2 = map(sorted_dirch, 1)

#calculate the mean average precision for each model
map3 = map(sorted_dirch, 2)

In [25]:
with open("tfidf.txt", 'w') as file:
  for q in data_dict:
    for vals in sorted_cosine[int(q) ]:
      file.write(f"{q} {vals}\n")


with open("okapi bm25.txt", 'w') as file:
  for q in data_dict:
    for vals in sorted_okapi[int(q) ]:
      file.write(f"{q} {vals}\n")


with open("Dirichlet Smoothing.txt", 'w') as file:
  for q in data_dict:
    for vals in sorted_dirch[int(q) ]:
      file.write(f"{q} {vals}\n")